In [1]:
import pandas as pd
import numpy as np

三种方式
.loc
①单个label
②列表索引
③切片
④布尔数组
⑤以①～④为返回值的单参数函数
.iloc
①整数(0到行数-1)
②整数列表索引
③整数切片
④布尔数组
⑤以①～④为返回值的单参数函数
[]
①对于序列，label/元素位置索引元素
②对于数据框，column/column列表索引列，用切片索引行，同时索引可使用[..][..]

In [2]:
df = pd.DataFrame({'A':['A0','A1','A2','A3'],'B':['B0','B1','B2','B3']})
s = pd.Series([5,2,7,8],index=list('abcd'),name='S')

s[[1,3]]
s[['b','d']]

df.loc[:,['B','A']] = df[['A','B']]
df.loc[:,['B','A']] = df[['A','B']].to_numpy()
df[['B','A']] = df[['A','B']]

In [3]:
# 利用属性方法访问列
s.b
df.B
# 利用属性方法修改/添加元素
s.b = 6
#s.e = 1 # 操作无效，但不会报错
df.A = list('WXYZ')
df['C'] = [0,1,2,3]
#df.C = [0,1,2,3] # 0.21.0后的版本，这样写是非法的

In [4]:
#[]的切片
s[1] #s['b']
s[-1] 
s[1:] #s['b':]
s[:1] #s[:'b']
s[::-1]
s[2::-1] #s['c'::-1]
s[2::-2] #s['c'::-2]
s[::] = list('WXYZ')
df = pd.DataFrame({'A':['A0','A1','A2','A3']
                  ,'B':['B0','B1','B2','B3']
                  ,'C':['C0','C1','C2','C3']})
df = df.set_index('B')
df[2:]
df['B2':]
df[['A','C']][2:]
df[2:][['A','C']]
#df[[0],['A']] #报错，格式不规范，这是类似于loc方法的格式

,A,C
B,,
B2,A2,C2
B3,A3,C3


In [5]:
#剩下两个区别在于，loc是值索引，iloc是位置索引
#使用loc
s.loc['b']
s.loc['b':]
s.loc['b':] = 0
df.loc[['B0','B1'],:]
df.loc[::-1,::-1]
#使用iloc(全是整数切片，不可能出现'...'这种)
s = pd.Series([5,6,7,8],index=list('abcd'),name='S')
s.iloc[:2]
s.iloc[2]
s.iloc[2] = 'a'
df.iloc[:2,:1]
df.iloc[:2,:1]

,A
B,
B0,A0
B1,A1


In [6]:
#用函数方法索引(返回内容必须是上述四者之一)
df.loc[lambda df:df.A != 'A1']
df.iloc[lambda x:[1,2]]
df.loc[lambda x:['B0','B3']] 
#df.iloc[lambda x:['B0','B3']] #报错，因为使用的是iloc
df[lambda df: df.columns[0]] #用[]方法
df.A.loc[lambda s:s!='A0'] #注意，传入的是loc前面的东西
#df.A.iloc[lambda s:s!='A0'] #报错，因为使用的是iloc

B
B1    A1
B2    A2
B3    A3
Name: A, dtype: object

In [7]:
#s.index[1] = 'a1' #报错，索引不能直接修改，正确方法如下(df.columns[1] = 'a'报错也是同样的原理)
new_index = s.index.tolist()
new_index[1] = 'a1'
s.index = new_index
#缺失型索引
#s.loc[['d','e']] #产生警告，对于不存在的索引，应当用reindex()方法：
s.reindex(['d','e'])
#如果想要把超出索引的剔除，可以使用intersection
s.loc[s.index.intersection(['d','e'])]
#下面更改行索引，产生重复项
new_index = s.index.tolist()
new_index[1] = 'a'
s.index = new_index
#s.reindex(['d','e']) #报错，重复项不能使用reindex

In [14]:
#抽样
s = pd.Series(list('abcdefghijklmnopqrstuvwxyz'))
s.sample() #默认抽一个
s.sample(n=10)
s.sample(frac=0.3) #抽30%的样本
#默认不放回抽样，如果放回抽样则设置replace参数
s.sample(n=5, replace=True) #可能有重复项
#不等概率抽样
pro = [0.1,0.3,0.4,0.2]
s = pd.Series(list('abcd'))
s.sample(n=2,weights=pro)
#如果权重不是sum为1，那么会归一化处理
pro = [3,5,2,4]
s.sample(n=2,weights=pro)
#抽列
df = pd.DataFrame({'A':['A0','A1','A2','A3']
                  ,'B':['B0','B1','B2','B3']
                  ,'C':['C0','C1','C2','C3']})
df.sample(n=2, axis=1)
#用random_state设置随机数种子
df.sample(n=2,axis=1,random_state=1)

,A,C
0,A0,C0
1,A1,C1
2,A2,C2
3,A3,C3


In [20]:
#利用loc和[]增加新行/列
s[4] = 'e'
df.loc[:,'D'] = df['A']
df

,A,B,C,D
0,A0,B0,C0,A0
1,A1,B1,C1,A1
2,A2,B2,C2,A2
3,A3,B3,C3,A3


In [22]:
#单元素索引的高效实现:at,iat
%timeit df.at[0,'A'] = 0
%timeit df.loc[0,'A'] = 0
%timeit df.iat[0,0] = 0
%timeit df.iloc[0,0] = 0

9.45 µs ± 68.7 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
210 µs ± 1.33 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
10.6 µs ± 63.9 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
205 µs ± 1.02 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [38]:
#布尔索引，其中“|”代表or操作，“&”代表and操作，“~”代表not操作
s = pd.Series(range(-5, 5))
s[s>0]
s[(s < -1) | (s > 0.5)]
s[~(s < 0)]
df = pd.DataFrame({'a': ['one', 'one', 'two', 'three', 'two', 'one', 'six'],
                    'b': ['x', 'y', 'y', 'x', 'y', 'x', 'x'],
                    'c': np.random.randn(7)})
df[[x.startswith('t') for x in df['a']]]
#上面更高效的写法，利用map函数：
df[df['a'].map(lambda x: x.startswith('t'))]
#现在布尔索引结合loc索引可以做许多事情
df.loc[df['a'].map(lambda x: x.startswith('t')) & (df['b'] == 'x'),'c'::-2]
#关于map函数的补充：
df['a'].map(lambda x:print(x)) #从打印内容看出如果用是对每一个元素的循环
df['a'].map(lambda x:'good') #直接输出内容，只不过上面使用了输出的是布尔值

one
one
two
three
two
one
six


0    good
1    good
2    good
3    good
4    good
5    good
6    good
Name: a, dtype: object

In [52]:
#isin方法
#(1)针对Series
s.isin([2, 4, 6]) #返回布尔值
s[s.isin([-10,-5,0,5,10])]
s.index.isin([0,3,6,10])
s[s.index.isin([0,3,6,10])]
#多级索引的isin，可以设置level
s_mi = pd.Series(np.arange(6),
                 index=pd.MultiIndex.from_product([[0, 1], ['a', 'b', 'c']]))
s_mi.iloc[s_mi.index.isin([(1, 'a'), (2, 'b'), (0, 'c')])]
s_mi.iloc[s_mi.index.isin(['a', 'c', 'e'], level=1)]
#(2)针对DataFrame
df = pd.DataFrame({'vals': [1, 2, 3, 4], 'ids': ['a', 'b', 'f', 'n'],
                   'ids2': ['a', 'n', 'c', 'n']})
values = {'ids': ['a', 'b'], 'vals': [1, 3]}
df.isin(values)
#(3)与any或all方法的结合，能够完成许多筛选
values = {'ids': ['a', 'b'], 'ids2': ['a', 'c'], 'vals': [1, 3]}
df[df.isin(values).all(1)] #1代表横向/跨列判断

,vals,ids,ids2
0,1,a,a


In [ ]:
#where方法和query方法
